In [75]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, mean_squared_error, r2_score

In [3]:
df = pd.read_csv('./Data/p.csv')
df_numeric = df.select_dtypes(include=['number']).copy()
df_numeric['player_name'] = df['full_name']

Agora, tendo em vista que o Random Forest precisa de uma variável alvo, precisamos defini-la

In [4]:
# Criar a variável alvo: se o jogador marcou algum gol (gols > 0)
df_numeric['gols_binario'] = (df_numeric['goals_overall'] > 0).astype(int)

In [5]:
# Definir X (features) e y (target)
X = df_numeric.drop(['goals_overall', 'gols_binario', 'player_name'], axis=1)
y = df_numeric['gols_binario']

In [7]:
# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Treinar o modelo Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [10]:
# Prever probabilidades de marcar um gol
probabilidades = model.predict_proba(X_test)[:, 1]

In [ ]:
# Criar DataFrame com nomes e probabilidades
resultado = pd.DataFrame({'Jogador': df_numeric.loc[X_test.index, 'player_name'], 'Probabilidade de Gol': probabilidades})

# Ordenar os jogadores por probabilidade
resultado_ordenado = resultado.sort_values(by='Probabilidade de Gol', ascending=False)
resultado_ordenado.head(10)

Com isso, verificou-se a necessidade de filtrar por dois times específicos que estariam jogando, pois o modelo prevê quais jogadores têm mais chance de fazer gol no geral.

In [16]:
team_a = "Corinthians"
team_b = "Juventude"

df_teams = df_numeric[(df['Current Club'] == team_a) | (df['Current Club'] == team_b)].copy()

In [ ]:
# Verificar se temos jogadores dos dois times
print(f"Jogadores do {team_a} e {team_b}: {df_teams.shape[0]}")

In [18]:
# Definir as features (X) para esses jogadores, removendo a coluna 'gols_binario' e outras não numéricas
X_teams = df_teams.drop(['goals_overall', 'gols_binario', 'player_name'], axis=1)

# Usar o modelo treinado para prever as probabilidades de gol para os jogadores desses times
probabilidades_teams = model.predict_proba(X_teams)[:, 1]

In [21]:
# Criar DataFrame com os nomes dos jogadores e suas respectivas probabilidades de gol
resultado_teams = pd.DataFrame({
    'Jogador': df_teams['player_name'], 
    'Probabilidade de Gol': probabilidades_teams
})

# Ordenar os jogadores pela probabilidade de marcar gol
resultado_teams_ordenado = resultado_teams.sort_values(by='Probabilidade de Gol', ascending=False)

In [ ]:
resultado_teams_ordenado.head(10)

Com o modelo foi possível identifiar as features mais importantes utilizadas por ele, então removemos as que tinham valores menores ou iguais a 0

In [24]:
importances = model.feature_importances_

feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances
})

In [29]:
features_importance_zero_or_less = feature_importances[feature_importances['Importance'] <= 0]
nomes_features_zero_or_less = features_importance_zero_or_less['Feature'].tolist()

df = df.drop(columns=nomes_features_zero_or_less)

Após isso, calculamos a acurácia e plotamos a matriz de confusão:

In [ ]:
# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcular a acurácia
acuracia = accuracy_score(y_test, y_pred)
print(f"Acurácia dos testes: {acuracia:.4f}")
print(f"{acuracia*100:.2f}%")

In [ ]:
# Calcular a matriz de confusão
matriz_confusao = confusion_matrix(y_test, y_pred)

# Criar um DataFrame para a matriz de confusão
df_matriz_confusao = pd.DataFrame(
    matriz_confusao,
    index=['Classe 0', 'Classe 1'],  
    columns=['Predito Classe 0', 'Predito Classe 1'] 
)

# Exibir a matriz de confusão com uma visualização
disp = ConfusionMatrixDisplay(confusion_matrix=matriz_confusao,
                              display_labels=['Sem Gol', 'Gol']) 
disp.plot(cmap='Blues')


Com isso, observamos uma acurácia de quase 100%, o que sigifica que o modelo provavelmente não está funcionando corretamente. Assim, tentamos uma outra abordagem, desta vez usando Random Forest de regressão ao invés de classificação.

In [54]:
# Definir X (features) e y (target: número de gols)
X_teams = df_teams.drop(['goals_overall', 'player_name'], axis=1)
y_teams = df_teams['goals_overall']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_teams, y_teams, test_size=0.3, random_state=42)

In [ ]:
# Treinar o modelo Random Forest Regressor
model = RandomForestRegressor(n_estimators=10, random_state=42)
model.fit(X_train, y_train)

In [59]:
predicoes_teams = model.predict(X_test)

In [61]:
# Criar DataFrame com os nomes dos jogadores e suas respectivas previsões de gols
resultado_teams = pd.DataFrame({
    'Jogador': df_teams.loc[X_test.index, 'player_name'], 
    'Previsão de Gols': predicoes_teams
})

In [63]:
resultado_teams_filtrado = resultado_teams[resultado_teams['Previsão de Gols'] >= 1]

# Ordenar os jogadores pela previsão de gols
resultado_teams_ordenado = resultado_teams_filtrado.sort_values(by='Previsão de Gols', ascending=False)

In [ ]:
resultado_teams_ordenado

In [ ]:
mse = mean_squared_error(y_test, predicoes_teams)
r2 = r2_score(y_test, predicoes_teams)

print(f'Mean Squared Error: {mse}')
print(f'R² Score: {r2}')

In [77]:
# Normalizar as previsões para intervalo [0, 1] usando uma transformação sigmoide
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Aplicar a função sigmoide para transformar previsões em probabilidades
probabilities = sigmoid(predicoes_teams)

In [ ]:
resultado_probabilidades = pd.DataFrame({
    'Jogador': df_teams.loc[X_test.index, 'player_name'], 
    'Probabilidade': probabilities
})

resultado_probabilidades

In [ ]:
top_10 = resultado_probabilidades.sort_values(by='Probabilidade', ascending=False).head(10)
top_10